In [412]:
import numpy as np
import cv2
np.set_printoptions(precision=2)

import ipywidgets.widgets as widgets
from IPython.display import display

In [413]:
class MarkerTracker:
    a = 1
    b = 1

    def __init__(self,colour):
        self.marker_list = []
        self.conc_map = np.full(grid_size,0,dtype=np.float64)
        self.colour = colour
    
    def addMarker(self,pos):
        self.marker_list.append(pos)
        self.generateConcMap()

    def generateConcMap(self):
        conc_map = np.full(grid_size,0,dtype=np.float64)

        for i_y,i_x in np.ndindex(grid_size):
        # this iterates over every single cell in the grid
            conc = 0
            for marker in self.marker_list:
                # this iterates over the markers
                dist = ((marker[1] - i_y)**2 + (marker[0] - i_x)**2)**0.5 + 1
                
                conc += a/(dist**(b))

            conc_map[i_y,i_x] = conc

        self.conc_map=conc_map
    

In [414]:
grid_size = (15,20)
marker_lookup = {"A":MarkerTracker((0,255,0)),
                 "B":MarkerTracker((255,0,0))}
marker_lookup["A"].addMarker((1,2))
marker_lookup["A"].addMarker((5,5))

marker_lookup["B"].addMarker((1,5))
marker_lookup["B"].addMarker((4,2))

In [415]:
class Walker:
    job_dict = {"scout":{"sens":"A","dep":"B"}}
    direction_lookup = [(-1,-1),
                        (0,-1),
                        (1,-1),
                        (1,0),
                        (1,1),
                        (0,1),
                        (-1,1),
                        (-1,0)]

    def __init__(self, pos, dir, job="scout"):
        self.pos = pos # tuple of 2 ints
        self.dir = dir # int between 0 and 7
        self.job = job # some key from job_dict

    def calculateMove(self):
        left_dir = (self.dir-1) % 8
        right_dir = (self.dir+1) % 8

        left_pos = (self.pos[0]+self.direction_lookup[left_dir][0],self.pos[1]+self.direction_lookup[left_dir][1])
        front_pos = (self.pos[0]+self.direction_lookup[self.dir][0],self.pos[1]+self.direction_lookup[self.dir][1])
        right_pos = (self.pos[0]+self.direction_lookup[right_dir][0],self.pos[1]+self.direction_lookup[right_dir][1])

        print("I am in position:",self.pos)
        print("the left is",left_pos)
        print("the front is",front_pos)
        print("the right is",right_pos)
        conc_map = marker_lookup[self.job_dict[self.job]["sens"]]

        left_conc = conc_map[left_pos[1],left_pos[0]]
        front_conc = conc_map[front_pos[1],front_pos[0]]
        right_conc = conc_map[right_pos[1],right_pos[0]]
        print(round(left_conc,4),round(front_conc,4),round(right_conc,4))

    def commitMove(self):
        pass


In [416]:
walker1 = Walker((1,1),2,"scout")
walker2 = Walker((3,3),7,"carrier")
walker_list = [walker1,walker2]
walker1.calculateMove()

I am in position: (1, 1)
the left is (1, 0)
the front is (2, 0)
the right is (2, 1)


In [417]:
cell_scale = 64
bottom_padding_size = 128
canvas_x_size = grid_size[1] * cell_scale
canvas_y_size = grid_size[0] * cell_scale + bottom_padding_size
canvas = np.full((int(canvas_y_size),int(canvas_x_size),3),150)

display_markers = []

def arrow_cache_builder():
    arrow_cache = []
    angles = [-45,0,45,90,135,180,225,270]
    arrow_scale = 0.6
    original = np.asarray([[0,0],[1,-1]]) * (cell_scale/2) * arrow_scale
    for i,deg in enumerate(angles):
        rad = deg * (np.pi/180)
        c = np.cos(rad)
        s = np.sin(rad)
        rot_mat = np.asarray([[c,-s],[s,c]])

        transformed = np.matmul(rot_mat,original) + np.full((2,2),cell_scale/2)
        end = (transformed[0,0],transformed[1,0])
        tip = (transformed[0,1],transformed[1,1])
        arrow_cache += [{"tip":tip,"end":end}]

    return arrow_cache
    

def interp(vec1, vec2,t,clamp=False):
    if clamp:
        t = max(0,min(1,t))
    return vec1 * (1-t) + vec2 * t

def col_interp(col1,col2,t,clamp=False):
    vec1 = np.asarray(col1)
    vec2 = np.asarray(col2)
    res = interp(vec1,vec2,t,clamp)

    res = np.clip(res,0,255)
    col = (int(res[0]),int(res[1]),int(res[2]))
    return col

def blend_colour(col_list):
    a = 2
    
    total = [0,0,0]
    for col in col_list:
        total[0] += col[0]**a
        total[1] += col[1]**a
        total[2] += col[2]**a

    total = (int((total[0]/len(col_list))**(1/a)),int((total[1]/len(col_list))**(1/a)),int((total[2]/len(col_list))**(1/a)))
    return total

def drawCells(canvas):
    base_cell_bg_colour = (128,128,128)

    for i_y,i_x in np.ndindex(grid_size):
        cell_tl = (int(i_x*cell_scale),int(i_y*cell_scale))
        cell_br = (cell_tl[0] + cell_scale-1,cell_tl[1] + cell_scale-1)

        col_list = []
        if(len(display_markers) != 0):
            for label in display_markers:
                #iterate through the labels of all the markers we're displaying
                marker_tracker = marker_lookup[label]

                # this is temporary; it needs to generate a colour based on the concentration and base colours of all of the markers we're trying to display
                conc=marker_tracker.conc_map[i_y,i_x]
                temp_col = col_interp(base_cell_bg_colour,marker_tracker.colour,conc,True)
                col_list += [temp_col]
            
            cell_bg_colour = blend_colour(col_list)
        else:
            cell_bg_colour = base_cell_bg_colour
        canvas = cv2.rectangle(canvas, cell_tl, cell_br, cell_bg_colour, -1)
        cell_edge_colour = col_interp((0,0,0),cell_bg_colour,0.6,True)
        canvas = cv2.rectangle(canvas, cell_tl, cell_br, cell_edge_colour, thickness=int(cell_scale * 0.05))

    return canvas

def drawWalkers(canvas):
    colour_dict = {"scout":(0,0,0),
                   "carrier":(0,0,200)}
    for walker in walker_list:
        pos = walker.pos
        dir = walker.dir
        colour = colour_dict[walker.job]

        temp = arrow_cache[dir]
        tip = temp["tip"]
        end = temp["end"]

        tip_x = int(pos[0]*cell_scale + tip[0])
        tip_y = int(pos[1]*cell_scale + tip[1])
        end_x = int(pos[0]*cell_scale + end[0])
        end_y = int(pos[1]*cell_scale + end[1])

        canvas = cv2.arrowedLine(canvas, (end_x,end_y), (tip_x,tip_y), colour, thickness=int(cell_scale * 0.05), tipLength = 0.2) 
    return canvas

arrow_cache = arrow_cache_builder()
canvas = drawCells(canvas)
canvas = drawWalkers(canvas)

In [418]:
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg',value)[1])

image_widget = widgets.Image(format='jpeg', width=512, height=512)
image_widget.value = bgr8_to_jpeg(canvas)
display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…